In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [2]:
class MultiHeadAttention(nn.Module):
    def __init__(self, vector_dimension, num_heads):
        super().__init__()
        assert vector_dimension % num_heads == 0, "vector_dimension must be divisible by num_heads"

        self.vector_dimension = vector_dimension
        self.num_heads = num_heads
        self.dimention_each_head = vector_dimension // num_heads

        self.W_q = nn.Linear(vector_dimension, vector_dimension)
        self.W_k = nn.Linear(vector_dimension, vector_dimension)
        self.W_v = nn.Linear(vector_dimension, vector_dimension)
        self.W_o = nn.Linear(vector_dimension, vector_dimension)

    def split_heads(self, x):
        batch_size, seq_len, vector_dimension = x.size()
        return x.view(batch_size, self.num_heads, seq_len, self.dimention_each_head)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.dimention_each_head)
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)
        attention_probs = torch.softmax(attention_scores, dim=-1)
        output = torch.matmul(attention_probs, V)
        return output

    def combine_heads(self, x):
        batch_size, _, seq_len, _ = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.vector_dimension)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_q(K))
        V = self.split_heads(self.W_q(V))

        attention_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attention_output))
        return output
        

In [3]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, vector_dimention, latent_dimention):
        super().__init__()
        self.linear1 = nn.Linear(vector_dimention, latent_dimention)
        self.linear2 = nn.Linear(latent_dimention, vector_dimention)
        self.activation = nn.ReLU()

    def forward(self, x):
        output = self.linear1(x)
        output = self.activation(output)
        output = self.linear2(output)
        return output
        

In [4]:
class PositionEncoding(nn.Module):
    def __init__(self, vector_dimention, max_seq_len):
        super().__init__()
        pe = torch.zeros(max_seq_len, vector_dimention)
        position = torch.arange(0, max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, vector_dimention, 2).float() * (-(math.log(10000.0) / vector_dimention)))

        pe[:, ::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, : x.size(1)]
        

In [5]:
class Encoder(nn.Module):
    def __init__(self, vector_dimention, num_heads, latent_dimention, dropout):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(vector_dimention, num_heads)
        self.feed_forward = PositionWiseFeedForward(vector_dimention, latent_dimention)

        self.norm1 = nn.LayerNorm(vector_dimention)
        self.norm2 = nn.LayerNorm(vector_dimention)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        output_attention = self.multi_head_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(output_attention))
        output_ff = self.feed_forward(x)
        x = self.norm2(x + self.dropout(output_ff))
        return x     
        

In [6]:
class Decoder(nn.Module):
    def __init__(self, vector_dimention, num_heads, latent_dimention, dropout):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(vector_dimention, num_heads)
        self.multi_head_cross_attention = MultiHeadAttention(vector_dimention, num_heads) 
        self.feed_forward = PositionWiseFeedForward(vector_dimention, latent_dimention)

        self.norm1 = nn.LayerNorm(vector_dimention)
        self.norm2 = nn.LayerNorm(vector_dimention)
        self.norm3 = nn.LayerNorm(vector_dimention)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encode_output, trg_mask, src_mask):
        output_self_attention = self.multi_head_attention(x, x, x, trg_mask)
        x = self.norm1(x + self.dropout(output_self_attention))
        
        output_cross_attention = self.multi_head_cross_attention(x, encode_output, encode_output, src_mask)
        x = self.norm2(x + self.dropout(output_cross_attention))
        
        output_ff = self.feed_forward(x)
        x = self.norm3(x + self.dropout(output_ff))

        return x

In [25]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, vector_dimention, num_heads, num_layers, latent_dimention, max_seq_len, dropout):
        super().__init__()
        self.position_encoder = PositionEncoding(vector_dimention, max_seq_len)
        self.encoder_embeddings = nn.Embedding(src_vocab_size, vector_dimention)
        self.decoder_embeddings = nn.Embedding(trg_vocab_size, vector_dimention)
        
        self.encoder_layers = nn.ModuleList([Encoder(vector_dimention, num_heads, latent_dimention, dropout) for _ in range(0, num_layers)])
        self.decoder_layers = nn.ModuleList([Decoder(vector_dimention, num_heads, latent_dimention, dropout) for _ in range(0, num_layers)])

        self.linear_final = nn.Linear(vector_dimention, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, trg):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        trg_mask = (trg != 0).unsqueeze(1).unsqueeze(3)
        seq_length = trg.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        trg_mask = trg_mask & nopeak_mask
        return src_mask.to(device), trg_mask.to(device)
    
    def forward(self, input_embedding, target_embedding):
        pos_input_emb = self.dropout(self.position_encoder(self.encoder_embeddings(input_embedding)))
        pos_trg_emb = self.dropout(self.position_encoder(self.encoder_embeddings(target_embedding)))

        src_mask, trg_mask = self.generate_mask(input_embedding, target_embedding)

        encod_output = pos_input_emb
        for encod_layer in self.encoder_layers:
            encod_output = encod_layer(encod_output, src_mask)

        decod_output = pos_trg_emb
        for decod_layer in self.decoder_layers:
            decod_output = decod_layer(decod_output, encod_output, trg_mask, src_mask)

        output = self.linear_final(decod_output)
        
        return output

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
src_vocab_size = 5000
trg_vocab_size = 5000
vector_dimention = 512
num_heads = 8
num_layers = 6
latent_dimention = 2048
max_seq_length = 100
dropout = 0.25
batch_size = 32
epochs = 100

transformer = Transformer(src_vocab_size, trg_vocab_size, vector_dimention, num_heads, num_layers, latent_dimention, max_seq_length, dropout).to(device)

# Generate random sample data
src_data = torch.randint(1, src_vocab_size, (batch_size, max_seq_length)).to(device)  # (batch_size, seq_length)
trg_data = torch.randint(1, trg_vocab_size, (batch_size, max_seq_length)).to(device)  # (batch_size, seq_length)

In [28]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(transformer.parameters(), lr=0.001, betas=(0.9, 0.99), eps=1e-9)
transformer.train()
for epoch in range(1, epochs + 1):
    criterion.zero_grad()
    predicts = transformer(src_data, trg_data[:, :-1])
    loss = criterion(predicts.contiguous().view(-1, trg_vocab_size), trg_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}")
    

Epoch: 1, Loss: 8.680929183959961
Epoch: 2, Loss: 8.23397159576416
Epoch: 3, Loss: 7.727494239807129
Epoch: 4, Loss: 7.244029521942139
Epoch: 5, Loss: 6.978521347045898
Epoch: 6, Loss: 6.529621124267578
Epoch: 7, Loss: 6.010624885559082
Epoch: 8, Loss: 5.696918964385986
Epoch: 9, Loss: 5.446065425872803
Epoch: 10, Loss: 5.271342754364014
Epoch: 11, Loss: 5.129522323608398
Epoch: 12, Loss: 5.26650333404541
Epoch: 13, Loss: 5.7097673416137695
Epoch: 14, Loss: 5.9030890464782715
Epoch: 15, Loss: 5.882598400115967
Epoch: 16, Loss: 6.7095842361450195
Epoch: 17, Loss: 6.675469875335693
Epoch: 18, Loss: 6.799657344818115
Epoch: 19, Loss: 7.327692985534668
Epoch: 20, Loss: 7.554989814758301
Epoch: 21, Loss: 7.484623432159424
Epoch: 22, Loss: 7.351357936859131
Epoch: 23, Loss: 7.322887897491455
Epoch: 24, Loss: 7.3069748878479
Epoch: 25, Loss: 7.314591884613037
Epoch: 26, Loss: 7.387606143951416
Epoch: 27, Loss: 7.465312957763672
Epoch: 28, Loss: 7.413875102996826
Epoch: 29, Loss: 7.19204235076